### Brain extraction (Skull stripping)

Brain extraction tool: https://pypi.org/project/deepbrain/

In [ ]:
import shutil
import gc
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from deepbrain import Extractor

import sys
sys.path.append('../')
from utils.myUtils import *
from utils.brainImageUtils import *
from mySettings import get_path_dict_by_datatype,get_preprocessing_type_list,get_preprocess_normalize_path_dict,get_experiment_key

In [ ]:
'''
Run brain Extraction by deepbrain package.
See https://github.com/iitzco/deepbrain/blob/master/bin/deepbrain-extractor.
'''
def extract_brain_mask(image_path, save_brain_mask_path, save_visualize_basic_path, show_results=False):
    print('\nBegin to extract the brain of {}.'.format(image_path))
    
    # read data
    image=nib.load(image_path)
    affine = image.affine
    image = image.get_fdata()
    
    #extract brain
    ext = Extractor()
    prob = ext.run(image)  # `prob` will be a 3d numpy image containing probability of being brain tissue for each of the voxels in `img`
    mask = prob > 0.5 
    
    
    #Save mask as nifti
    brain_mask = (1 * mask).astype(np.uint8)
    brain_mask = nib.Nifti1Image(brain_mask, affine)
    nib.save(brain_mask, save_brain_mask_path)
    print('\n Finsh saving the mask to {}.'.format(save_brain_mask_path))
    
    #show the brain extraction results
    if show_results:
        show_brain_extraction_results(image_path, save_brain_mask_path, save_visualize_basic_path)
        
        
'''
Show brain MRI data and the extracted brain mask.
'''
def show_brain_extraction_results(image_path, mask_path, save_visualize_path, axis="transverse"):

    #read data
    image=nib.load(image_path).get_fdata()
    mask=nib.load(mask_path).get_fdata()
    
    #show the 3D image slice by slice
    axis_index=get_axis_index()
    number_of_slices=image.shape[axis_index[axis]]
    for index in range(0, number_of_slices):
        image_slice=get_image_slice(image, axis, index)
        mask_slice=get_image_slice(mask, axis, index)
        
        plt.figure(figsize=(15,5))
        sub_fig=plt.subplot(1,3, 1)
        sub_fig.set_title("image", fontsize=12)
        plt.axis('off')
        plt.imshow(image_slice, aspect='auto',cmap='gray')
        
        sub_fig=plt.subplot(1,3,2)
        sub_fig.set_title("mask", fontsize=12)
        plt.axis('off')
        plt.imshow(mask_slice*255, aspect='auto',cmap='gray')
        
        sub_fig=plt.subplot(1,3,3)
        sub_fig.set_title("mask over image", fontsize=12)
        plt.axis('off')
        plt.imshow(image_slice, aspect='auto',cmap='gray')
        plt.imshow(mask_slice, alpha=0.1, aspect='auto')

        plt.subplots_adjust(left=0.03, bottom=0.03, right=0.97, top=0.9,  wspace=0.03, hspace=0)   
        basename=os.path.basename(image_path)
        save_path= save_visualize_path+'/'+ basename[:-7]+'_'+str(index)+'.png'
        plt.savefig(save_path)
        #plt.show()
        
        # Clear the current axes.
        plt.cla() 
        # Clear the current figure.
        plt.clf() 
        # Closes all the figure windows.
        plt.close('all')
        gc.collect()

In [ ]:
'''
Perform brain mask extraction for images in a given folder.
'''
def perform_brain_mask_extraction(registed_image_base_path, save_brain_mask_base_path, save_visualize_base_path):
    registed_image_paths=get_filenames(registed_image_base_path, '.nii.gz')
    print("\n Extracting brain masks! \n There are {} images in {}.".format(len(registed_image_paths), registed_image_base_path))
    
    num_images=len(registed_image_paths)
    i=0
    for registed_image_path in registed_image_paths:
        i=i+1
        print("=============brain extraction: {}/{}========".format(i, num_images))
        save_brain_mask_path=save_brain_mask_base_path+'/'+os.path.basename(registed_image_path)
        save_visualize_path=save_visualize_base_path+'/'+os.path.basename(registed_image_path)[:-7]
        mkdir(save_visualize_path)
        extract_brain_mask(registed_image_path, save_brain_mask_path, save_visualize_path)

### Main: brain extraction

In [ ]:
def main_brain_extraction():
    #basic settings
    preprocess_type_list=get_preprocessing_type_list()
    experiment_key_list=get_experiment_key()
    
    #extract brain mask according to defined preprocess type.
    for preprocess_type in preprocess_type_list:   
        print("====================================preprocess_type={}================================================".format(preprocess_type))
        registered_image_path_dict=get_preprocess_normalize_path_dict(path_type='registered_images', preprocess_type=preprocess_type)
        brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='brain_mask', preprocess_type=preprocess_type)
        check_brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='check_brain_mask', preprocess_type=preprocess_type)
        for experiment_key in experiment_key_list:
            print("============={}=============".format(experiment_key))
            registered_image_dir=registered_image_path_dict[experiment_key]
            save_brain_mask_dir=brain_mask_path_dict[experiment_key]
            save_check_brain_mask_dir=check_brain_mask_path_dict[experiment_key]
            perform_brain_mask_extraction(registered_image_dir, save_brain_mask_dir, save_check_brain_mask_dir)

In [ ]:

main_brain_extraction()


### Copy brain mask of the fixed image to the moving images.

When doing the intensity normalization, since the images have been registered, we need to keep consistent brain mask of the registered images. So we use the brain mask of the fixed images as the brain mask of the moving images.

In [ ]:
def perform_unify_brain_mask(brain_mask_dir, consistent_brain_mask_dir, registration_info_txt):
    registration_info_dict=parse_registration_info_txt(registration_info_txt)
    for patient, registration_info in registration_info_dict.items():
        
        transform=registration_info["transform"]
        fixed_image_name=registration_info["fixed_image"]
        moving_image_name_list=registration_info["moving_image_list"]
        
        orig_fixed_image_path=os.path.join(brain_mask_dir, fixed_image_name+ '.nii.gz')
        for image_name in [fixed_image_name]+moving_image_name_list:
            
            dest_image_path=os.path.join(consistent_brain_mask_dir, image_name+ '.nii.gz')
            shutil.copy(orig_fixed_image_path, dest_image_path)
            
            
    
def main_consistent_brain_masks():
    #basic settings
    preprocess_type_list=get_preprocessing_type_list()
    experiment_key_list=get_experiment_key()
    registration_info_txt_dict=get_path_dict_by_datatype(data_type='registration_info')
    
    #extract brain mask according to defined preprocess type.
    for preprocess_type in preprocess_type_list:   
        print("====================================preprocess_type={}================================================".format(preprocess_type))
        brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='brain_mask', preprocess_type=preprocess_type)
        consistent_brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='consistent_brain_mask', preprocess_type=preprocess_type)
        for experiment_key in experiment_key_list:
            print("============={}=============".format(experiment_key))
            brain_mask_dir=brain_mask_path_dict[experiment_key]
            consistent_brain_mask_dir=consistent_brain_mask_path_dict[experiment_key]
            registration_info_txt=registration_info_txt_dict[experiment_key]
            
            perform_unify_brain_mask(brain_mask_dir, consistent_brain_mask_dir, registration_info_txt)



In [ ]:
main_consistent_brain_masks()

### Brain extraction for the unregistered images, for illustration of the image histogram

In [ ]:
def main_brain_extraction_unregisteredImages():
    #basic settings
    preprocess_type_list=get_preprocessing_type_list()
    experiment_key_list=get_experiment_key()
    
    #extract brain mask according to defined preprocess type.
    for preprocess_type in preprocess_type_list:   
        print("====================================preprocess_type={}================================================".format(preprocess_type))
        registered_image_path_dict=get_preprocess_normalize_path_dict(path_type='preprocessed_data', preprocess_type=preprocess_type)
        brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='brain_mask_unregistered', preprocess_type=preprocess_type)
        check_brain_mask_path_dict=get_preprocess_normalize_path_dict(path_type='check_brain_mask_unregistered', preprocess_type=preprocess_type)
        for experiment_key in experiment_key_list:
            print("============={}=============".format(experiment_key))
            registered_image_dir=registered_image_path_dict[experiment_key]
            save_brain_mask_dir=brain_mask_path_dict[experiment_key]
            save_check_brain_mask_dir=check_brain_mask_path_dict[experiment_key]
            perform_brain_mask_extraction(registered_image_dir, save_brain_mask_dir, save_check_brain_mask_dir)

In [ ]:
main_brain_extraction_unregisteredImages()